In [ ]:
from ultralytics import SAM
import torch
import tifffile as tf
import matplotlib.pyplot as plt
import numpy as np  
from skimage.metrics import structural_similarity as ssim  
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Utilisation de: {device}")

: 

In [ ]:
image = tf.imread('../data/Romane_Martin_urne_sature_10-4.tif')

In [ ]:
#print(image.shape)
#print(image)

In [ ]:
plt.figure(figsize=(12, 10))
plt.imshow(image[800, :, :],cmap='gray')
plt.title('Coupe Z = 800')
#plt.imsave('coupez800.png', image[800, :, :], cmap='gray')
plt.show()
slice_z = image[800, :, :]  # ← AJOUTER CETTE LIGNE

In [ ]:
# Load a model
model = SAM("sam_b.pt")

# Display model information (optional)
model.info()

In [ ]:
# ============ Configuration ============
START_SLICE = 400
END_SLICE = 500
NUM_SLICES = END_SLICE - START_SLICE
urne_np = np.zeros(image.shape)
print(f"Analyse de {NUM_SLICES} coupes (Z={START_SLICE} à Z={END_SLICE-1})")

# ============ Fonction pour extraire masque binaire ============
def get_binary_mask(model, image_slice):
    """
    Segmente une coupe et retourne l'image binaire (objets = 255, fond = 0)
    """
    # Sauvegarder temporairement (SAM nécessite un fichier)
    temp_file = "temp_slice.png"
    plt.imsave(temp_file, image_slice, cmap='gray')
    
    # Segmentation
    result = model(temp_file)
    
    # Extraire masques
    if result[0].masks is not None:
        masks = result[0].masks.data.cpu().numpy()
        
        # Créer masque combiné
        combined_mask = np.zeros(image_slice.shape, dtype=bool)
        for mask in masks:
            combined_mask = np.logical_or(combined_mask, mask)
        
        # Image binaire
        binary_image = np.zeros(image_slice.shape, dtype=np.uint8)
        binary_image[combined_mask] = 255
        
        return binary_image, len(masks)
    else:
        return np.zeros(image_slice.shape, dtype=np.uint8), 0

# ============ Traitement des coupes ============
num_objects_list = []

print("\nTraitement en cours...")

# Traiter la première coupe

#prev_binary = None
#for z in range(START_SLICE, END_SLICE):
for z in range(image.shape[0]):
    if z % 10 == 0:  # Afficher progression tous les 10
        print(f"  Coupe {z}/{image.shape[0]}")
    
    current_slice = image[z, :, :]
    current_binary, num_objects = get_binary_mask(model, current_slice)
    #_, num_objects = get_binary_mask(model, current_slice)
    urne_np[z]=current_binary
    num_objects_list.append(num_objects)


print(f"\n✓ Traitement terminé!")

tf.imwrite('temp.tif', urne_np, photometric='rgb')

# Plot 2: Nombre d'objets détectés
z_all = np.arange(0, image.shape[0])
axes[1].plot(z_all, num_objects_list, 'g-', linewidth=2, marker='s', markersize=3)
axes[1].axhline(y=np.mean(num_objects_list), color='r', linestyle='--', 
                label=f'Moyenne = {np.mean(num_objects_list):.1f}')
axes[1].set_xlabel('Index de coupe Z', fontsize=12)
axes[1].set_ylabel('Nombre d\'objets', fontsize=12)
axes[1].set_title('Nombre d\'objets détectés par coupe', fontsize=14, fontweight='bold')
axes[1].grid(True, alpha=0.3)
axes[1].legend()

plt.tight_layout()

In [ ]:


# ============ Traitement des coupes ============
ssim_values = []
print("\nTraitement en cours...")

# Traiter la première coupe
step = 2
#prev_binary = None
prev_slice = None
for z in range(0, image.shape[0] - step, step):
    if z % 10 == 0:  # Afficher progression tous les 10
        print(f"  Coupe {z}/{END_SLICE-1}")
    
    current_slice = image[z, :, :]
    #current_binary, num_objects = get_binary_mask(model, current_slice)
    #_, num_objects = get_binary_mask(model, current_slice)
    
    num_objects_list.append(num_objects)
    
    # Calculer SSIM avec la coupe précédente
    #if prev_binary is not None:
    if prev_slice is not None:
        #ssim_value = ssim(prev_binary, current_binary, data_range=255)
        ssim_value = ssim(prev_slice, current_slice, data_range=255)
        ssim_values.append(ssim_value)
    
    #prev_binary = current_binary
    prev_slice = current_slice

print(f"\n✓ Traitement terminé!")

# ============ Nettoyage ============
import os
if os.path.exists("temp_slice.png"):
    os.remove("temp_slice.png")

# ============ Affichage des résultats ============
z_indices = np.arange(START_SLICE + 1, END_SLICE)

fig, axes = plt.subplots(2, 1, figsize=(14, 10))

# Plot 1: SSIM
axes[0].plot(z_indices, ssim_values, 'b-', linewidth=2, marker='o', markersize=3)
axes[0].axhline(y=np.mean(ssim_values), color='r', linestyle='--', 
                label=f'Moyenne = {np.mean(ssim_values):.4f}')
axes[0].set_xlabel('Index de coupe Z', fontsize=12)
axes[0].set_ylabel('SSIM', fontsize=12)
axes[0].set_title('Similarité structurelle (SSIM) entre coupes consécutives', fontsize=14, fontweight='bold')
axes[0].grid(True, alpha=0.3)
axes[0].legend()
axes[0].set_ylim([0, 1])


# ============ SAUVEGARDER LES GRAPHIQUES ============
# Graphique combiné (2 plots)
#output_file = f'ssim_analysis_Z{START_SLICE}-{END_SLICE}.png'
#plt.savefig(output_file, dpi=300, bbox_inches='tight')
#print(f"\n📊 Graphiques combinés sauvegardés: {output_file}")

# SSIM seul
fig_ssim, ax_ssim = plt.subplots(figsize=(14, 5))
ax_ssim.plot(z_indices, ssim_values, 'b-', linewidth=2, marker='o', markersize=3)
ax_ssim.axhline(y=np.mean(ssim_values), color='r', linestyle='--', 
                label=f'Moyenne = {np.mean(ssim_values):.4f}')
ax_ssim.set_xlabel('Index de coupe Z', fontsize=12)
ax_ssim.set_ylabel('SSIM', fontsize=12)
ax_ssim.set_title('Similarité structurelle (SSIM) entre coupes consécutives', fontsize=14, fontweight='bold')
ax_ssim.grid(True, alpha=0.3)
ax_ssim.legend()
ax_ssim.set_ylim([0, 1])
plt.tight_layout()
plt.savefig(f'ssim_only_Z{START_SLICE}-{END_SLICE}.png', dpi=300, bbox_inches='tight')
print(f"📈 SSIM seul sauvegardé: ssim_only_Z{START_SLICE}-{END_SLICE}.png")
plt.close(fig_ssim)

# Nombre d'objets seul
"""
fig_obj, ax_obj = plt.subplots(figsize=(14, 5))
ax_obj.plot(z_all, num_objects_list, 'g-', linewidth=2, marker='s', markersize=3)
ax_obj.axhline(y=np.mean(num_objects_list), color='r', linestyle='--', 
               label=f'Moyenne = {np.mean(num_objects_list):.1f}')
ax_obj.set_xlabel('Index de coupe Z', fontsize=12)
ax_obj.set_ylabel('Nombre d\'objets', fontsize=12)
ax_obj.set_title('Nombre d\'objets détectés par coupe', fontsize=14, fontweight='bold')
ax_obj.grid(True, alpha=0.3)
ax_obj.legend()
plt.tight_layout()
plt.savefig(f'objects_count_Z{START_SLICE}-{END_SLICE}.png', dpi=300, bbox_inches='tight')
print(f"📊 Nombre d'objets seul sauvegardé: objects_count_Z{START_SLICE}-{END_SLICE}.png")
plt.close(fig_obj)

plt.show()
"""
# ============ Statistiques ============
print("\n" + "="*50)
print("=== STATISTIQUES SSIM ===")
print("="*50)
print(f"SSIM moyen:     {np.mean(ssim_values):.4f}")
print(f"SSIM min:       {np.min(ssim_values):.4f} (coupe Z={z_indices[np.argmin(ssim_values)]})")
print(f"SSIM max:       {np.max(ssim_values):.4f} (coupe Z={z_indices[np.argmax(ssim_values)]})")
print(f"Écart-type:     {np.std(ssim_values):.4f}")

print("\n" + "="*50)
"""
print("=== STATISTIQUES OBJETS ===")
print("="*50)
print(f"Objets moyen:   {np.mean(num_objects_list):.1f}")
print(f"Objets min:     {np.min(num_objects_list)} (coupe Z={z_all[np.argmin(num_objects_list)]})")
print(f"Objets max:     {np.max(num_objects_list)} (coupe Z={z_all[np.argmax(num_objects_list)]})")

# Identifier les grandes variations
threshold = 0.05
large_variations = np.where(np.array(ssim_values) < (1 - threshold))[0]
if len(large_variations) > 0:
    print("\n" + "="*50)
    print(f"⚠️  GRANDES VARIATIONS (SSIM < {1-threshold:.2f}):")
    print("="*50)
    for idx in large_variations[:10]:
        print(f"  Z={z_indices[idx]}: SSIM = {ssim_values[idx]:.4f}")

print("\n🎉 Analyse terminée!")
"""

In [ ]:
# ============ VISUALISATION DE TROIS COUPES SUCCESSIVES ============
Z_TO_SHOW = [799, 800, 801]
fig, axes = plt.subplots(len(Z_TO_SHOW), 4, figsize=(16, 4 * len(Z_TO_SHOW)))

for i, z in enumerate(Z_TO_SHOW):
    img = image[z, :, :]
    
    # Segmentation de la coupe
    temp_file = f"temp_slice_{z}.png"
    plt.imsave(temp_file, img, cmap='gray')
    result = model(temp_file)
    
    # Extraction des masques
    if result[0].masks is not None:
        masks = result[0].masks.data.cpu().numpy()
        combined_mask = np.zeros(img.shape, dtype=bool)
        for mask in masks:
            combined_mask |= mask
        
        binary_image = np.zeros(img.shape, dtype=np.uint8)
        binary_image[combined_mask] = 255
        
        labeled_image = np.zeros(img.shape, dtype=np.int32)
        
        # Créer l'overlay coloré pour la segmentation
        overlay = np.zeros((*img.shape, 3), dtype=np.uint8)
        np.random.seed(42)  # Pour des couleurs reproductibles
        
        for label_id, mask in enumerate(masks, start=1):
            labeled_image[mask] = label_id
            # Générer une couleur aléatoire pour chaque objet
            color = np.random.randint(0, 255, 3)
            overlay[mask] = color
    else:
        binary_image = np.zeros(img.shape, dtype=np.uint8)
        labeled_image = np.zeros(img.shape, dtype=np.int32)
        masks = []
        overlay = np.zeros((*img.shape, 3), dtype=np.uint8)
    
    # Affichage
    ax_row = axes[i] if len(Z_TO_SHOW) > 1 else axes
    
    # 1️⃣ Image originale
    ax_row[0].imshow(img, cmap='gray')
    ax_row[0].set_title(f"Image originale\nZ={z}")
    ax_row[0].axis("off")
    
    # 2️⃣ Segmentation Ultralytics (SEULEMENT l'overlay sur l'image)
    ax_row[1].imshow(img, cmap='gray')
    if overlay.sum() > 0:  # Si des objets ont été détectés
        ax_row[1].imshow(overlay, alpha=0.5)
    ax_row[1].set_title(f"Segmentation Ultralytics\n{len(masks)} objets")
    ax_row[1].axis("off")
    
    # 3️⃣ Objets uniquement (binaire)
    ax_row[2].imshow(binary_image, cmap='gray')
    ax_row[2].set_title(f"Objets uniquement (binaire)\n{len(masks)} objets")
    ax_row[2].axis("off")
    
    # 4️⃣ Matrice de labels
    im = ax_row[3].imshow(labeled_image, cmap='nipy_spectral')
    ax_row[3].set_title(f"Matrice de labels\n{len(masks)} objets")
    ax_row[3].axis("off")
    fig.colorbar(im, ax=ax_row[3], fraction=0.046, pad=0.04)
    
    # Nettoyer le fichier temporaire
    os.remove(temp_file)

plt.tight_layout()
plt.show()